# How to evaluate a RAG application

This example uses [Langchain](https://www.langchain.com) and [Giskard](https://github.com/Giskard-AI/giskard) to evaluate the quality of a RAG application.

In [1]:
!pip install langchain
!pip install langchain-openai
!pip install langchain_experimental
!pip install langchain_community
!pip install docarray
!pip install pydantic==1.10.8
!pip install python-dotenv
!pip install ruff
!pip install bs4
!pip install ipytest
!pip install giskard[llm]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install --quiet openai python-dotenv

In [6]:
import os
from dotenv import load_dotenv
from google.colab import userdata

load_dotenv()

#OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
OPENAI_API_KEY = "COLOQUE AQUI SUA API KEY DA OPEN AI" #Link: https://platform.openai.com/api-keys
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
MODEL = "gpt-3.5-turbo"

## Scrape the Website and Split the Content

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

loader = WebBaseLoader("https://postech.fiap.com.br/curso/ia-para-devs/")
documents = loader.load_and_split(text_splitter)
documents

[Document(page_content='IA PARA DEVSFECHARDeixe aqui seus dados para garantir seu benefícioENVIAR*Campos obrigatóriosLI E CONCORDO COM OS TERMOS DA POLÍTICA DE PRIVACIDADE.Para continuar é necessário marcar essa opçãoENVIAR0%Inscreva-se01FIAP + AluraSobre02CursosDev / Cyber / Data / Tech & Business03Hands-onAprenda fazendo04ComunidadeA maior do Brasil05Unidades + PolosSP / BH / POA / RJ / REC06Home07Perguntas frequentes08Contato09Matrícula;PÓS TECHIA PARA DEVS360 horas - 10 mesesINSCREVA-SETurma de marçopós-graduação 100% digital e hands-onIA PARA DEVSDESENVOLVA SOLUÇÕES COM AS TÉCNICAS MAIS AVANÇADAS DE INTELIGÊNCIA ARTIFICIAL E MACHINE LEARNING.Nesta pós-graduação especialmente pensada para devs, você vai ampliar suas oportunidades de carreira ao criar sistemas e aplicativos que resolvem desafios complexos utilizando técnicas de IA.Explore o universo do Machine Learning na nuvem, domine o Processamento de Linguagem Natural, aplique Algoritmos Genéticos, desvende o potencial das LLMs 

## Load the Content in a Vector Store

In [7]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    documents, embedding=OpenAIEmbeddings()
)

## Create a Knowledge Base

Let's start by loading the content in a pandas DataFrame.

In [8]:
import pandas as pd

df = pd.DataFrame([d.page_content for d in documents], columns=["text"])
df.head(10)

,text
0,IA PARA DEVSFECHARDeixe aqui seus dados para g...
1,Language Models) e incorpore as Inteligências ...
2,e aplicaçõesEvolução computacional e otimizaçã...
3,de Indivíduos e Codificação de GenesOperadores...
4,of Thought com Base Científica em LLMsDesenvol...
5,e regulamentação de proteção de dados (LGPD)An...
6,"assíncronas, 100% hands-on, contam com vídeos ..."
7,vez mais complexos e ainda contar com um case ...
8,"códigos de machine learning,aprenda e comparti..."
9,"Paulista, 1106 / Edifício Paulista, 1100 3º, ..."


We can now create a Knowledge Base using the DataFrame we created before.

In [9]:
from giskard.rag import KnowledgeBase

knowledge_base = KnowledgeBase(df)

/usr/local/lib/python3.10/dist-packages/giskard/core/validation.py:31: RuntimeWarning: fields may not start with an underscore, ignoring "_args"
  validated_func = validate_arguments(func, config={"arbitrary_types_allowed": True})
/usr/local/lib/python3.10/dist-packages/giskard/core/validation.py:31: RuntimeWarning: fields may not start with an underscore, ignoring "_kwargs"
  validated_func = validate_arguments(func, config={"arbitrary_types_allowed": True})


## Generate the Test Set

In [10]:
from giskard.rag import generate_testset

testset = generate_testset(
    knowledge_base,
    num_questions=60,
    agent_description="Um chatbot respondendo perguntas sobre o site da FIAP de IA para Desenvolvedores",
)

INFO:giskard.rag:Finding topics in the knowledge base.
INFO:giskard.rag:Computing Knowledge Base embeddings.
/usr/local/lib/python3.10/dist-packages/umap/umap_.py:2433: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(
INFO:giskard.rag:Found 1 topics in the knowledge base.


Generating questions:   0%|          | 0/60 [00:00<?, ?it/s]

Let's display a few samples from the test set.

In [11]:
test_set_df = testset.to_pandas()

for index, row in enumerate(test_set_df.head(3).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference answer: {row[1]['reference_answer']}")
    print("Reference context:")
    print(row[1]['reference_context'])
    print("******************", end="\n\n")


Question 1: What are some of the sections available on the FIAP website?
Reference answer: Some sections available on the FIAP website are Fotos, Vídeos, Prêmios e Reconhecimentos, Parcerias Estratégicas, HUB FIAP, Unidades, COMISSÃO PRÓPRIA DE AVALIAÇÃO, CPA 2018, CPA 2017, CPA 2016, CPA 2015, Fale Conosco, Trabalhe Conosco, Política de Privacidade, Gerencie seus cookies.
Reference context:
Document 18: documento deve ser enviado antes do início das aulas.11 98170-002811 3385-8010FIAP 2022.Todos os direitos reservados.A FIAPA FIAPFotosVídeosPrêmios e ReconhecimentosParcerias EstratégicasHUB FIAPUnidadesCOMISSÃO PRÓPRIA DE AVALIAÇÃOCPA 2018CPA 2017CPA 2016CPA 2015LINKS ÚTEISFale ConoscoTrabalhe ConoscoPolítica de PrivacidadeGerencie seus cookiesUNIDADESAclimaçãoPaulista00 - 00
******************

Question 2: What is the duration of the IA PARA DEVS post-graduate program?
Reference answer: The IA PARA DEVS post-graduate program lasts for 360 hours over a period of 10 months.
Reference c

Let's now save the test set to a file:

In [12]:
testset.save("test-set.jsonl")

## Prepare the Prompt Template

In [13]:
from langchain.prompts import PromptTemplate

template = """
Responda à pergunta com base no contexto abaixo. Se você não pode
responda à pergunta, responda "Não sei".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Responda à pergunta com base no contexto abaixo. Se você não pode
responda à pergunta, responda "Não sei".

Context: Here is some context

Question: Here is a question



## Create the RAG Chain

Create a retriever from the Vector Store that will allow us to get the top similar documents to a given question.

In [14]:
retriever = vectorstore.as_retriever()
retriever.get_relevant_documents("O que é o curso de IA para Devs?")

[Document(page_content='IA PARA DEVSFECHARDeixe aqui seus dados para garantir seu benefícioENVIAR*Campos obrigatóriosLI E CONCORDO COM OS TERMOS DA POLÍTICA DE PRIVACIDADE.Para continuar é necessário marcar essa opçãoENVIAR0%Inscreva-se01FIAP + AluraSobre02CursosDev / Cyber / Data / Tech & Business03Hands-onAprenda fazendo04ComunidadeA maior do Brasil05Unidades + PolosSP / BH / POA / RJ / REC06Home07Perguntas frequentes08Contato09Matrícula;PÓS TECHIA PARA DEVS360 horas - 10 mesesINSCREVA-SETurma de marçopós-graduação 100% digital e hands-onIA PARA DEVSDESENVOLVA SOLUÇÕES COM AS TÉCNICAS MAIS AVANÇADAS DE INTELIGÊNCIA ARTIFICIAL E MACHINE LEARNING.Nesta pós-graduação especialmente pensada para devs, você vai ampliar suas oportunidades de carreira ao criar sistemas e aplicativos que resolvem desafios complexos utilizando técnicas de IA.Explore o universo do Machine Learning na nuvem, domine o Processamento de Linguagem Natural, aplique Algoritmos Genéticos, desvende o potencial das LLMs 

We can now create our chain.

In [15]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)

Let's make sure the chain works by testing it with a simple question.

In [16]:
chain.invoke({"question": "O que é o curso de IA para Devs?"})

'O curso de IA para Devs é uma pós-graduação 100% digital e hands-on que visa desenvolver soluções com as técnicas mais avançadas de inteligência artificial e machine learning, ampliando as oportunidades de carreira ao criar sistemas e aplicativos que resolvem desafios complexos utilizando técnicas de IA.'

## Avaliando o modelo no conjunto de testes

Precisamos criar uma função que invoque a cadeia com uma pergunta específica e retorne a resposta.

In [17]:
def answer_fn(question, history=None):
    return chain.invoke({"question": question})

Agora podemos usar a função avaliar() para avaliar o modelo no conjunto de teste. Esta função irá comparar as respostas da cadeia com as respostas de referência no conjunto de teste.

In [18]:
from giskard.rag import evaluate

report = evaluate(answer_fn, testset=testset, knowledge_base=knowledge_base)

Asking questions to the agent:   0%|          | 0/60 [00:00<?, ?it/s]

Correctness evaluation:   0%|          | 0/60 [00:00<?, ?it/s]

Let now display the report.

Here are the five components of our RAG application:

* **Generator**: This is the LLM used in the chain to generate the answers.
* **Retriever**: This is the retriever that fetches relevant documents from the knowledge base according to a query.
* **Rewriter**: This is a component that rewrites the user query to make it more relevant to the knowledge base or to account for chat history.
* **Router**: This is a component that filters the query of the user based on his intentions.
* **Knowledge Base**: This is the set of documents given to the RAG to generate the answers.

In [19]:
display(report)

GENERATOR,70.0% The Generator is the LLM inside the RAG to generate the answers.,70.0%
RETRIEVER,75.0% The Retriever fetches relevant documents from the knowledge base according to a user query.,75.0%
REWRITER,56.67% The Rewriter modifies the user query to match a predefined format or to include the context from the chat history.,56.67%
ROUTING,100.0% The Router filters the query of the user based on his intentions (intentions detection).,100.0%
KNOWLEDGE_BASE,100.0% The knowledge base is the set of documents given to the RAG to generate the answers. Its scores is computed differently than the other components: it is the difference between the maximum and minimum correctness score across all the topics of the knowledge base.,100.0%


In [20]:
report.to_html("report.html")

We can display the correctness results organized by question type.

In [21]:
report.correctness_by_question_type()

,correctness
question_type,
complex,0.7
conversational,0.7
distracting element,0.5
double,0.5
simple,1.0
situational,0.8


We can also display the specific failures.

In [22]:
report.get_failures()

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason
id,,,,,,,,
923737d3-43dd-4357-8414-dd5a643cc0b1,Could you specify the subjects that are being ...,The 7th and 8th months of the course cover Dat...,Document 4: of Thought com Base Científica em ...,[],"{'question_type': 'complex', 'seed_document_id...",Não sei.,False,The agent did not provide any information abou...
fb5ce4d4-27d6-4d12-a579-27eaf809b0c9,Could you elaborate on the specific advantages...,"Upon enrolling in the Pós Tech program, you ge...","Document 9: Paulista, 1106 / Edifício Paulista...",[],"{'question_type': 'complex', 'seed_document_id...","Ao se inscrever no programa Pós Tech, você ter...",False,The agent's answer does not mention the access...
141e4014-a383-474b-b529-2ccd61db7b76,What are the main domains of knowledge that th...,The key areas of preparation in the DEVTECH TO...,Document 1: Language Models) e incorpore as In...,[],"{'question_type': 'complex', 'seed_document_id...",Não sei.,False,The agent's answer 'Não sei' which translates ...
5a0b308f-1c9a-47cc-9dd8-47b6c991a46c,What are the required documents for enrollment...,The necessary documents for enrollment include...,"Document 16: do Brasil, entrar em contato pelo...",[],"{'question_type': 'distracting element', 'seed...",The required documents for enrollment at FIAP ...,False,The agent's answer is incorrect because it doe...
deeb099c-4f96-476a-bd04-ab28ba0a16ca,Considering that I have completed my graduatio...,The DEVTECH TOOLS course will prepare you to a...,Document 1: Language Models) e incorpore as In...,[],"{'question_type': 'distracting element', 'seed...",Não sei.,False,The agent's answer 'Não sei' (I don't know) do...
87d7e543-2ebf-4c3d-8e5a-8a60add53f0b,Considering that the live attendance is not ma...,The advanced Machine Learning section covers t...,Document 2: e aplicaçõesEvolução computacional...,[],"{'question_type': 'distracting element', 'seed...",Some of the topics covered in the advanced Mac...,False,The agent's answer does not match the ground t...
6ef39b59-86f3-4292-a395-4df500cc231f,Considering the Pós Tech course includes a pro...,"Yes, the Pós Tech course includes one in-perso...",Document 7: vez mais complexos e ainda contar ...,[],"{'question_type': 'distracting element', 'seed...",Não sei.,False,The agent's answer 'Não sei' which translates ...
8f044264-bf98-4a4b-8a1e-3c3f07b621aa,Is there a specific process or requirement for...,People with a completed higher education degre...,"Document 15: Bairro do Recife - Recife, PE.11O...",[],"{'question_type': 'distracting element', 'seed...",Não sei.,False,The agent's answer 'Não sei' (I don't know) do...
096126ab-174e-444f-be65-45382be4d03a,"As a recent graduate from Brazil, I'm interest...",The necessary documents for enrollment include...,"Document 16: do Brasil, entrar em contato pelo...",[],"{'question_type': 'situational', 'seed_documen...",Para se matricular no programa IA para Devs da...,False,"The agent's answer is in Portuguese, while the..."


## Creating a Test Suite

We can create a test suite and use it to compare different models.

Load the test set from disk.

In [23]:
from giskard.rag import QATestset

testset = QATestset.load("test-set.jsonl")

Create a Test Suite from the test set.

In [24]:
test_suite = testset.to_test_suite("Conjunto de testes escolares de IA")

We need a function that takes a DataFrame of questions, invokes the chain with each question, and returns the answers.

In [25]:
import giskard


def batch_prediction_fn(df: pd.DataFrame):
    return chain.batch([{"question": q} for q in df["question"].values])

We can now create a Giskard Model object to run our test suite.

In [26]:
giskard_model = giskard.Model(
    model=batch_prediction_fn,
    model_type="text_generation",
    name="Modelo de perguntas e respostas da FIAP",
    description="Este modelo responde a perguntas sobre o site da FIAP.",
    feature_names=["question"],
)

INFO:giskard.models.automodel:Your 'prediction_function' is successfully wrapped by Giskard's 'PredictionFunctionModel' wrapper class.


Let's now run the test suite using the model we created before.

In [27]:
test_suite_results = test_suite.run(model=giskard_model)

INFO:giskard.datasets.base:Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
ERROR:root:An error happened during test execution for test: TestsetCorrectnessTest
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/giskard/core/suite.py", line 573, in run
    result = test_partial.giskard_test(**test_params).execute()
  File "/usr/local/lib/python3.10/dist-packages/giskard/registry/giskard_test.py", line 192, in execute
    return configured_validate_arguments(self.test_fn)(*self.args, **self.kwargs)
  File "pydantic/decorator.py", line 40, in pydantic.decorator.validate_arguments.validate.wrapper_function
  File "pydantic/decorator.py", line 134, in pydantic.decorator.ValidatedFunction.call
  File "pydantic/decorator.py", line 206, in pydantic.decorator.ValidatedFunction.execute
  File "/usr/local/lib/python3.10/dist-packages/giskard/testing/tests/llm/correctness.py", line 35, in test_llm_correctness
    eval_result = correctn

We can display the results.

In [ ]:
display(test_suite_results)

<TestSuiteResult (passed)>

## Integrating with Pytest

In [ ]:
import ipytest

We can now integrate our test suite with Pytest.

In [ ]:
%%ipytest

import pytest
from giskard.rag import QATestset
from giskard.testing.tests.llm import test_llm_correctness


@pytest.fixture
def dataset():
    testset = QATestset.load("test-set.jsonl")
    return testset.to_dataset()


@pytest.fixture
def model():
    return giskard_model


def test_chain(dataset, model):
    test_llm_correctness(model=model, dataset=dataset, threshold=0.5).assert_()

.2024-03-23 16:27:56,471 pid:46357 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
2024-03-23 16:27:56,472 pid:46357 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (60, 5) executed in 0:00:00.005269


.                                                                                           [100%]
========================================= warnings summary =========================================
../.venv/lib/python3.9/site-packages/_pytest/config/__init__.py:1276
  /Users/svpino/dev/llm/.venv/lib/python3.9/site-packages/_pytest/config/__init__.py:1276: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: anyio
    self._mark_plugins_for_rewrite(hook)

t_66406511b9d84eb38baa6b0a22141dd0.py::test_llm_correctness
  /Users/svpino/dev/llm/.venv/lib/python3.9/site-packages/_pytest/python.py:199: PytestReturnNotNoneWarning: Expected None, but t_66406511b9d84eb38baa6b0a22141dd0.py::test_llm_correctness returned To execute the test call "execute()" method, which will be an error in a future version of pytest.  Did you mean to use `assert` instead of `return`?
    warnings.warn(

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
2 passed, 2 warni